In [10]:
import pandas as pd
import ast
import matplotlib.pyplot as plt
import warnings
import math
import numpy as np
import seaborn as sns
warnings.filterwarnings("ignore")

%matplotlib inline

In [11]:
## Functions

# Funktion, um den Wattwert für einen Prozess aus der 'processes'-Liste zu extrahieren
def extract_wattage(process_list, target_process):
    process_list = ast.literal_eval(process_list)
    for process, wattage in process_list:
        if process == target_process:
            return wattage
    return None

# Funktion um aus allen Messungen (W) den Energie Gesamt Energie-Verbrauch (KwH) für einen Service zu berechnen
def calculate_energy_consumption(process_name,process_wattage_slice,time_delta_slice):
    total_ws = 0.0
    total_t = 0

    for i in range(1,len(process_wattage_slice)):
        dt = time_delta_slice.iloc[i] - time_delta_slice.iloc[i-1]
        average = (process_wattage_slice.iloc[i] + process_wattage_slice.iloc[i-1]) / 2
        total_ws += average * dt
        total_t += dt
    
    if math.isnan(total_ws):
        return 0.0
    
    return total_ws
    

In [12]:
# Load data
from glob import glob
runs = glob(f"data/*/*")
runs += glob(f"data_scale/*/*")
runs += glob(f"data_scale_rampup/*/*")

In [13]:
all = None
services = None
max_run = 2
for run in runs:
    exp = run.split('/')[1]
    run_id = int(run.split('/')[2])
    if run_id > max_run:
        continue
    try:
        workload = pd.read_csv(f"{run}/teastore_stats.csv").tail(1)[["Request Count", "Failure Count","Median Response Time","Average Response Time","Average Content Size"]].rename(
            columns={"Request Count": "requests", "Failure Count": "failures", "Median Response Time": "med. lat.", "Average Response Time": "avg. lat.", "Average Content Size": "avg. size"}
        )
        num_requests = workload["requests"].values[0]
        
        obversations = pd.read_csv(glob(f"{run}/measurements_*.csv")[0])
        obversations["time"] = pd.to_datetime(obversations["timestamp"])
        obversations["processes"] = obversations["processes"].apply(lambda x: ast.literal_eval(x))
        total_runtime = (obversations["time"].max()-obversations["time"].min()).total_seconds()
        X = obversations.explode("processes").dropna()
        X["process_name"] = X["processes"].apply(lambda x: x[0])
        X["wattage"] = X["processes"].apply(lambda x: x[1])
        process_stats = X.groupby("process_name")["process_wattage"].sum().reset_index().rename(columns={"process_wattage": "total_wattage"})
        process_stats["total_wattage"] =process_stats["total_wattage"]/1000000
        process_stats["avg_wattage"] = process_stats["total_wattage"]/total_runtime
        process_stats["avg_wattage_per_request"] = process_stats["total_wattage"]/num_requests

        run_data = [exp,run_id]+workload.values.tolist()[0]+[total_runtime,process_stats["total_wattage"].sum(),process_stats["process_name"].nunique()]
        run_data = pd.DataFrame([run_data],columns=["exp","run_id","requests","failures","med_lat","avg_lat","avg_payload","runtime","wattage","deployment_units"])

        process_stats["exp"] = exp
        if "scale" in run:
            process_stats["scale"] = exp.split("_")[-1]
            run_data["scale"] = exp.split("_")[-1]
            process_stats["name"] =  "_".join(exp.split("_")[0:-1])
            run_data["name"] = "_".join(exp.split("_")[0:-1])
        else:
            process_stats["scale"] = "none"
            process_stats["name"] = exp
            run_data["name"] = exp
            run_data["scale"] = "none"
        
        
        process_stats["run_id"] = run_id
        process_stats["instance"] = process_stats["process_name"].apply(lambda x:X[X["process_name"]==x]["instance"].unique()[0]) 
        process_stats["process_name_normal"] = process_stats["process_name"].apply(lambda x:x.split("-")[1])
        

        if all is None:
            all = run_data
            services = process_stats
        else:
            all = pd.concat([all,run_data])
            services = pd.concat([services,process_stats])
    except FileNotFoundError as e:
        continue
    #     
    #process_measures = X.groupby("process_name").rolling("60s",on="time")["process_wattage"].sum()
    #print(exp,run_id,"\n",workload, "\n",process_stats[["process_name","avg_wattage","avg_wattage_per_request"]], "\ntotal runtime:",total_runtime,"\nreq/s:",workload["requests"].values[0]/total_runtime,"\n\n")
    # plt.figure()
    # sns.lineplot(data=process_measures.reset_index(),x="time",y="process_wattage",hue="process_name")
    # plt.suptitle(f"{exp} - {run_id}")


FileNotFoundError: [Errno 2] No such file or directory: 'data_scale/obs_feature_object-storage_cpu/1/teastore_stats.csv'

In [ ]:
X = all.groupby(["exp"])[["requests","failures","runtime","wattage","deployment_units"]].sum()
X["mW/Req"] = X["wattage"]/(X["requests"]+X["failures"])*1000
X = X.sort_values(by="mW/Req")
X

,requests,failures,runtime,wattage,deployment_units,mW/Req
exp,,,,,,
jvm_jvm-impoove,45098.0,241.0,935.478550,242.464604,7,5.347815
baseline_linearwl_vanilla_cpu,39934.0,1323.0,956.673808,459.398998,13,11.135056
baseline_vanilla_cpu,36890.0,287.0,936.737191,425.725005,13,11.451301
jvm_linearwl_jvm-impoove_cpu,45745.0,2210.0,957.149886,638.481807,13,13.314186
jvm_jvm-impoove_cpu,31510.0,261.0,936.932412,581.315993,14,18.297063
baseline_linearwl_vanilla,488.0,488.0,945.373498,43.270891,7,44.334929
baseline_vanilla,432.0,413.0,935.656085,45.236502,7,53.534322
jvm_linearwl_jvm-impoove,303.0,75.0,945.528908,82.912198,7,219.344439


In [ ]:
services.groupby(["name","scale","process_name_normal"])["total_wattage"].sum()

name                       scale  process_name_normal
baseline_linearwl_vanilla  cpu    auth                    54.263222
                                  db                      54.263222
                                  image                  106.855764
                                  persistence             54.263222
                                  recommender             84.683702
                                  registry                19.434585
                                  webui                   85.635281
                           none   auth                    10.212643
                                  db                       3.828844
                                  image                    3.135615
                                  persistence             10.212643
                                  recommender              2.532888
                                  registry                 3.135615
                                  webui                   10.2